In [1]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import sqlite3
from sqlite3 import Error
import csv
import matplotlib.pyplot as plt
import seaborn as sns

import os
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE
from sklearn.metrics import r2_score

import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [2]:
def create_connection(db_file, delete_db=False):
    import os
    if delete_db and os.path.exists(db_file):
        os.remove(db_file)

    conn = None
    try:
        conn = sqlite3.connect(db_file)
        conn.execute("PRAGMA foreign_keys = 1")
    except Error as e:
        print(e)

    return conn


def create_table(conn, create_table_sql, drop_table_name=None):
    
    if drop_table_name: # You can optionally pass drop_table_name to drop the table. 
        try:
            c = conn.cursor()
            c.execute("""DROP TABLE IF EXISTS %s""" % (drop_table_name))
        except Error as e:
            print(e)
    
    try:
        c = conn.cursor()
        c.execute(create_table_sql)
    except Error as e:
        print(e)
        
def execute_sql_statement(sql_statement, conn):
    cur = conn.cursor()
    cur.execute(sql_statement)

    rows = cur.fetchall()

    return rows


In [3]:
filename = "fake_job_postings.csv"

In [4]:
with open(filename, encoding="utf-8") as file:
    raw_data = list(csv.reader(file, delimiter=","))
raw_data = raw_data[1:]
raw_data1 = [tuple(i) for i in raw_data]
# print(raw_data1)

In [5]:
# raw_data = []
# with open(filename) as file:
#     for line in file:
#         if not line.strip():
#             continue
#         raw_data.append(line.strip())

# lst = [i.split(',') for i in raw_data]
# lst = lst[1:]
# lst1 = [tuple(i) for i in lst]

In [6]:
conn.close()

NameError: name 'conn' is not defined

In [ ]:
db_file = 'raw_data.db'
import os
if os.path.exists(db_file):
    os.remove(db_file)
conn = create_connection(db_file)

create_table_sql1 = """
    CREATE TABLE IF NOT EXISTS [raw_data] (
        [job_id] integer not null primary key,
        [title] text,
        [location] text,
        [department] text,
        [salary_range] text,
        [company_profile] text,
        [description] text,
        [requirements] text,
        [benefits] text,
        [telecommuting] integer,
        [has_company_logo] integer,
        [has_questions] integer,
        [employment_type] text,
        [required_experience] text,
        [required_education] text,
        [industry] text,
        [function] text,
        [fraudulent] integer
    );
    """

def insert_raw_data(conn, values):
    sql = ''' INSERT INTO raw_data(job_id,title,location,
                department,salary_range,company_profile,description,
                requirements,benefits,telecommuting,has_company_logo,
                has_questions,employment_type,required_experience,
                required_education,industry,function,fraudulent) 
                VALUES(?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?) '''
    cur = conn.cursor()
    cur.execute(sql, values)
    return cur.lastrowid

with conn:
        create_table(conn, create_table_sql1)
        for values in raw_data1:
            insert_raw_data(conn,values)

In [ ]:
db_file_norm = 'normalized_data1.db'

conn1 = create_connection(db_file_norm)

In [ ]:
with conn:
    sql_statement = 'SELECT distinct company_profile from raw_data'
    value_profile = execute_sql_statement(sql_statement, conn)

create_table_sql2 = """
    CREATE TABLE IF NOT EXISTS [company] (
        [companyID] integer not null primary key,
        [company_profile] text
    );
    """

def insert_profile(conn, values):
    sql = ''' INSERT INTO company(company_profile) 
                VALUES(?) '''
    cur = conn.cursor()
    cur.execute(sql, values)
    return cur.lastrowid

   

with conn1:
    create_table(conn1, create_table_sql2)
    for values in value_profile:
        insert_profile(conn1,values)

In [ ]:
sql_statement = 'SELECT distinct company_profile, has_company_logo,count(job_id) from raw_data group by company_profile,has_company_logo order by company_profile'
df = pd.read_sql_query(sql_statement, conn)
display(df)

In [ ]:
l1 = df.iloc[2:,0].values
l1.shape

In [ ]:
np.unique(l1).shape

In [ ]:
sql_statement = 'SELECT distinct department, count(job_id) from raw_data group by department'
df = pd.read_sql_query(sql_statement, conn)
df.head(50)

- title - title
- department - distinct department
- salary - id, distinct salary(min max)
- description - description
- requirements - id, req
- benefits - id,benefits
- telecommuting
- has_company_logo
- has_questions
- employment_type - 
- required_experience
- required_education
- industry
- function
- fraudulent
- job_title_loc: job_id, title, locid, titleid, salaryid
- company table: companyid, company_profile, has_company_logo
- location table: locid,location - seggregated
- jobid - companyid,  ###company_profile
- 
- posting: jobid, description, requirement, benefits, telecommuting, has_questions,employment_type,required_experience,required_education,industry,function
- 

In [ ]:
with conn:
    sql_statement = 'SELECT distinct title from raw_data'
    value_profile = execute_sql_statement(sql_statement, conn)

create_table_sql2 = """
    CREATE TABLE IF NOT EXISTS [title] (
        [title] text not null primary key
    );
    """

def insert_profile(conn, values):
    sql = ''' INSERT INTO company(company_profile) 
                VALUES(?) '''
    cur = conn.cursor()
    cur.execute(sql, values)
    return cur.lastrowid

   

with conn1:
    create_table(conn1, create_table_sql2)
    for values in value_profile:
        insert_profile(conn1,values)

------ D&S

In [ ]:
with conn:
    sql_statement = 'SELECT distinct telecommuting from raw_data'
    value_telecommuting = execute_sql_statement(sql_statement, conn)
#     insert_tuple = []
#     for row in value_telecommuting:
#         if row[0] == 1:
#             insert_tuple.append((True,))
#         else :
#             insert_tuple.append((False,)) 
    
    
create_table_sql2 = """
    CREATE TABLE IF NOT EXISTS [Telecommuting] (
        [telecommutingId] integer not null primary key,
        [isTelecommuting] boolean
    );
    """

def insert_telecommuting(conn, values):
    sql = ''' INSERT INTO Telecommuting(isTelecommuting) 
                VALUES(?) '''
    cur = conn.cursor()
    cur.executemany(sql, values)
    return cur.lastrowid

   

with conn1:
    create_table(conn1, create_table_sql2,'Telecommuting')
    insert_telecommuting(conn1,value_telecommuting)
#     sql_statement = 'SELECT * from Telecommuting'
#     telecommuting = execute_sql_statement(sql_statement, conn1)

In [ ]:
insert_tuple

In [ ]:
value_telecommuting

In [ ]:
telecommuting

In [ ]:
with conn:
    sql_statement = 'SELECT distinct has_company_logo from raw_data'
    value_companylogo = execute_sql_statement(sql_statement, conn)
#     insert_tuple = []
#     for row in value_telecommuting:
#         if row[0] == 1:
#             insert_tuple.append((True,))
#         else :
#             insert_tuple.append((False,)) 
    
    
create_table_sql2 = """
    CREATE TABLE IF NOT EXISTS [Companylogo] (
        [LogoId] integer not null primary key,
        [HasLogo] boolean
    );
    """

def insert_companylogo(conn, values):
    sql = ''' INSERT INTO Companylogo(HasLogo) 
                VALUES(?) '''
    cur = conn.cursor()
    cur.executemany(sql, values)
    return cur.lastrowid

   

with conn1:
    create_table(conn1, create_table_sql2,'Companylogo')
    insert_companylogo(conn1,value_companylogo)
#     sql_statement = 'SELECT * from Companylogo'
#     logo = execute_sql_statement(sql_statement, conn1)

In [ ]:
logo

In [ ]:
with conn:
    sql_statement = 'SELECT distinct has_questions from raw_data'
    value_questions = execute_sql_statement(sql_statement, conn)
#     insert_tuple = []
#     for row in value_telecommuting:
#         if row[0] == 1:
#             insert_tuple.append((True,))
#         else :
#             insert_tuple.append((False,)) 
    
    
create_table_sql2 = """
    CREATE TABLE IF NOT EXISTS [HasQuestions] (
        [QuestionId] integer not null primary key,
        [HasQuestion] boolean
    );
    """

def insert_questions(conn, values):
    sql = ''' INSERT INTO HasQuestions(HasQuestion) 
                VALUES(?) '''
    cur = conn.cursor()
    cur.executemany(sql, values)
    return cur.lastrowid

   

with conn1:
    create_table(conn1, create_table_sql2,'HasQuestions')
    insert_questions(conn1,value_questions)
#     sql_statement = 'SELECT * from HasQuestions'
#     question = execute_sql_statement(sql_statement, conn1)

In [ ]:
question

In [ ]:
with conn:
    sql_statement = 'SELECT distinct employment_type from raw_data'
    value_employment_type = execute_sql_statement(sql_statement, conn)

create_table_sql2 = """
    CREATE TABLE IF NOT EXISTS [EmploymentTypes] (
        [EmploymentTypeId] integer not null primary key,
        [EmploymentType] Text
    );
    """

def insert_eType(conn, values):
    sql = ''' INSERT INTO EmploymentTypes(EmploymentType) 
                VALUES(?) '''
    cur = conn.cursor()
    cur.executemany(sql, values)
    return cur.lastrowid

   

with conn1:
    create_table(conn1, create_table_sql2,'EmploymentTypes')
    insert_eType(conn1,value_employment_type)
#     sql_statement = 'SELECT * from EmploymentTypes'
#     employment_type = execute_sql_statement(sql_statement, conn1)

In [ ]:
value_employment_type

In [ ]:
# df = pd.read_csv("fake_job_postings.csv")

In [ ]:
# df.head()

In [ ]:
# df.shape

In [ ]:
employment_type

In [ ]:
with conn:
    sql_statement = 'SELECT distinct required_experience from raw_data'
    value_required_experience = execute_sql_statement(sql_statement, conn)

    
create_table_sql2 = """
    CREATE TABLE IF NOT EXISTS [RequiredExperiences] (
        [RequiredExperienceId] integer not null primary key,
        [RequiredExperience] Text
    );
    """

def insert_req_experience(conn, values):
    sql = ''' INSERT INTO RequiredExperiences(RequiredExperience) 
                VALUES(?) '''
    cur = conn.cursor()
    cur.executemany(sql, values)
    return cur.lastrowid

   

with conn1:
    create_table(conn1, create_table_sql2,'RequiredExperience')
    insert_req_experience(conn1,value_required_experience)
#     sql_statement = 'SELECT * from RequiredExperiences'
#     experience_type = execute_sql_statement(sql_statement, conn1)

In [ ]:
experience_type

In [ ]:
with conn:
    sql_statement = 'SELECT distinct required_education from raw_data'
    value_required_education = execute_sql_statement(sql_statement, conn) 
    
create_table_sql2 = """
    CREATE TABLE IF NOT EXISTS [RequiredEducation] (
        [RequiredEducationId] integer not null primary key,
        [Education] Text
    );
    """

def insert_req_education(conn, values):
    sql = ''' INSERT INTO RequiredEducation(Education) 
                VALUES(?) '''
    cur = conn.cursor()
    cur.executemany(sql, values)
    return cur.lastrowid

   

with conn1:
    create_table(conn1, create_table_sql2,'Education')
    insert_req_education(conn1,value_required_education)
#     sql_statement = 'SELECT * from RequiredEducation'
#     required_education = execute_sql_statement(sql_statement, conn1)

In [ ]:
required_education

In [ ]:
with conn:
    sql_statement = 'SELECT distinct industry from raw_data'
    value_industry = execute_sql_statement(sql_statement, conn) 
    
create_table_sql2 = """
    CREATE TABLE IF NOT EXISTS [Industry] (
        [IndustryId] integer not null primary key,
        [IndustryName] Text
    );
    """

def insert_Industry(conn, values):
    sql = ''' INSERT INTO Industry(IndustryName) 
                VALUES(?) '''
    cur = conn.cursor()
    cur.executemany(sql, values)
    return cur.lastrowid

   

with conn1:
    create_table(conn1, create_table_sql2,'Industry')
    insert_Industry(conn1,value_required_education)
#     sql_statement = 'SELECT * from Industry'
#     industry = execute_sql_statement(sql_statement, conn1)

In [ ]:
industry

In [ ]:
with conn:
    sql_statement = 'SELECT distinct benefits from raw_data'
    value_benefits = execute_sql_statement(sql_statement, conn) 
    
create_table_sql2 = """
    CREATE TABLE IF NOT EXISTS [Benefits] (
        [BenefitsId] integer not null primary key,
        [BenefitsDescription] Text
    );
    """

def insert_Benefits(conn, values):
    sql = ''' INSERT INTO Benefits(BenefitsDescription) 
                VALUES(?) '''
    cur = conn.cursor()
    cur.executemany(sql, values)
    return cur.lastrowid

   

with conn1:
    create_table(conn1, create_table_sql2,'Benefits')
    insert_Benefits(conn1,value_benefits)
#     sql_statement = 'SELECT * from Benefits'
#     benefits = execute_sql_statement(sql_statement, conn1)
#     sql_statement = 'SELECT * from Benefits where BenefitsDescription=""'
#     benefits = execute_sql_statement(sql_statement, conn1)